In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model

In [3]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define paths
original_dataset_dir = '/Users/code/alzheimerDetect/OriginalDataset'
base_dir = '/Users/code/alzheimerDetect/split_dataset'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# Create directories for training and testing data
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Define categories
categories = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']

for category in categories:
    os.makedirs(os.path.join(train_dir, category), exist_ok=True)
    os.makedirs(os.path.join(test_dir, category), exist_ok=True)

    # List files in the category directory
    category_dir = os.path.join(original_dataset_dir, category)
    files = os.listdir(category_dir)
    
    # Split the data
    train_files, test_files = train_test_split(files, test_size=0.2, random_state=42)
    
    # Copy files to train directory
    for file in train_files:
        src = os.path.join(category_dir, file)
        dst = os.path.join(train_dir, category, file)
        shutil.copyfile(src, dst)
    
    # Copy files to test directory
    for file in test_files:
        src = os.path.join(category_dir, file)
        dst = os.path.join(test_dir, category, file)
        shutil.copyfile(src, dst)

print("Dataset split into training and testing sets.")

Dataset split into training and testing sets.


In [4]:
# Data Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1/255.0,
    validation_split=0.2  # 20% of the data will be used for validation
)

test_datagen = ImageDataGenerator(
    rescale=1/255.0
)

In [5]:
# Load training data and validation data from the training directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Resize images to 150x150 pixels
    batch_size=32,
    class_mode='categorical',  # Adjusted for multi-class classification
    subset='training'  # Set as training data
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',  # Adjusted for multi-class classification
    subset='validation'  # Set as validation data
)

# Load test data from the test directory
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'  # Adjusted for multi-class classification
)

Found 4096 images belonging to 4 classes.
Found 1023 images belonging to 4 classes.
Found 1281 images belonging to 4 classes.


In [6]:
# Load the VGG19 model with pretrained ImageNet weights, without the top classification layer
vgg = VGG19(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze all the layers in VGG19 so their weights don't get updated during training
for layer in vgg.layers:
    layer.trainable = False

# Add custom classification layers to the base VGG19 model
x = Flatten()(vgg.output)  # Flatten the output from VGG19
x = Dense(512, activation='relu')(x)  # Add a fully connected layer with 512 units
x = Dropout(0.5)(x)  # Add dropout to reduce overfitting
output = Dense(4, activation='softmax')(x)  # Final output layer for multi-class classification

# Create the full model by combining VGG19 base with the custom layers
model = Model(inputs=vgg.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

2024-09-22 17:39:19.015284: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-09-22 17:39:19.015334: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-09-22 17:39:19.015362: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-09-22 17:39:19.015596: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-22 17:39:19.015621: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
# Create an EarlyStopping callback
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

In [8]:
# Train the Model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[callback]
)

Epoch 1/20


2024-09-22 17:39:56.688028: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
/opt/anaconda3/envs/tf_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


128/128 ━━━━━━━━━━━━━━━━━━━━ 34s 249ms/step - accuracy: 0.4736 - loss: 1.6133 - val_accuracy: 0.5181 - val_loss: 0.9536
Epoch 2/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 31s 245ms/step - accuracy: 0.5400 - loss: 0.9578 - val_accuracy: 0.5533 - val_loss: 0.9115
Epoch 3/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 32s 246ms/step - accuracy: 0.5517 - loss: 0.9194 - val_accuracy: 0.5728 - val_loss: 0.8876
Epoch 4/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 32s 248ms/step - accuracy: 0.5931 - loss: 0.8580 - val_accuracy: 0.5718 - val_loss: 0.9227
Epoch 5/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 32s 250ms/step - accuracy: 0.5824 - loss: 0.8781 - val_accuracy: 0.5885 - val_loss: 0.8610
Epoch 6/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 32s 251ms/step - accuracy: 0.6193 - loss: 0.8441 - val_accuracy: 0.6041 - val_loss: 0.8604
Epoch 7/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 32s 250ms/step - accuracy: 0.5963 - loss: 0.8299 - val_accuracy: 0.6158 - val_loss: 0.8298
Epoch 8/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 32s 247ms/step - accuracy: 0.6197 - loss: 0.8068 - val

In [9]:
# Evaluate the Model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy:.4f}')

41/41 ━━━━━━━━━━━━━━━━━━━━ 9s 219ms/step - accuracy: 0.7360 - loss: 0.6226
Test accuracy: 0.7229


In [ ]:
# Save the Model
model.save('alzheimers_detector_model_with_originalDS_cnn_VGG_arch.h5')

In [ ]:
# Optionally, plot the training and validation accuracy and loss
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

# Define the path to the image
image_path = 'split_dataset/test/MildDemented/29 (4).jpg'

# Load the image
img = image.load_img(image_path, target_size=(150, 150))

# Display the image
plt.imshow(img)
plt.title("MRI Scan")
plt.axis('off')
plt.show()

# Preprocess the image
img_array = image.img_to_array(img)  # Convert the image to a numpy array
img_array = img_array / 255.0  # Rescale the image
img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to match the model's input shape

# Make a prediction
prediction = model.predict(img_array)

# Define class labels
class_labels = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']

# Get the predicted class
predicted_class = class_labels[np.argmax(prediction[0])]

print(f'The model predicts: {predicted_class} ({prediction[0][np.argmax(prediction[0])]:.4f})')

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

# Load the saved model
model = tf.keras.models.load_model('alzheimers_detector_model_with_originalDS_cnn_VGG_arch.h5')

# Define the path to the image
image_path = '/Users/arnabsahoo/Downloads/FinalPriject/Alzheimer_s Dataset/test/VeryMildDemented/26 (53).jpg'

# Load the image
img = image.load_img(image_path, target_size=(150, 150))

# Display the image
plt.imshow(img)
plt.title("MRI Scan")
plt.axis('off')
plt.show()

# Preprocess the image
img_array = image.img_to_array(img)  # Convert the image to a numpy array
img_array = img_array / 255.0  # Rescale the image
img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to match the model's input shape

# Make a prediction
prediction = model.predict(img_array)

# Define class labels
class_labels = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']

# Get the predicted class
predicted_class = class_labels[np.argmax(prediction[0])]

print(f'The model predicts: {predicted_class} ({prediction[0][np.argmax(prediction[0])]:.4f})')

In [ ]:
import matplotlib.pyplot as plt

# Plotting accuracy and loss for each batch
plt.figure(figsize=(12, 6))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(accuracy_per_batch, label='Test Accuracy')
plt.title('Model Accuracy per Batch')
plt.xlabel('Batch Number')
plt.ylabel('Accuracy')
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(loss_per_batch, label='Test Loss', color='red')
plt.title('Model Loss per Batch')
plt.xlabel('Batch Number')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()